## Import to Postgres

Load the prices and fundamentals into postgres.

```shell
DB_NAME=stocks pgfutter csv prices.csv
DB_NAME=stocks pgfutter csv  --fields "ticker_indicator,date,value" fundamentals.csv
```

Setup the prices table.

```sql
ALTER TABLE import.prices
SET SCHEMA public;

ALTER TABLE prices ALTER COLUMN "date" 
SET DATA TYPE date 
USING date::date;

CREATE INDEX idx_prices_date 
ON prices(date);
```

Setup the fundamentals materialized view.

```sql
ALTER TABLE import.fundamentals ALTER COLUMN "date" 
SET DATA TYPE date 
USING date::date;

CREATE MATERIALIZED VIEW fundamentals AS
SELECT split_part(ticker_indicator, '_', 1) as ticker, date, value as epsdil 
FROM import.fundamentals
WHERE ticker_indicator LIKE '%EPSDIL_MRT';

CREATE INDEX idx_fundamentals_date 
ON fundamentals(date);

CREATE INDEX idx_fundamentals_ticker 
ON fundamentals(ticker);
```

## Query and Export

Export the prices with fundamentals. This took about 40 minutes.

```sql
COPY 

(SELECT p.*, 

CASE p.epsdil::numeric
	WHEN 0 THEN 0
	ELSE p.adj_close::numeric / p.epsdil::numeric
END as pe

FROM (
	SELECT prices.adj_close, prices.date, prices.ticker,
	(
		SELECT fundamentals.epsdil
		FROM fundamentals
		WHERE prices.ticker = fundamentals.ticker
		AND prices.date >= fundamentals.date 
		ORDER BY fundamentals.date DESC
		LIMIT 1
	)
	FROM prices
	WHERE prices.adj_close IS NOT NULL AND prices.adj_close != ''
) p
WHERE p.epsdil IS NOT NULL AND p.epsdil != ''
)

TO 'prices.csv' (format CSV);
```